#Выбор семейства распределений в наивном байесе

В этом задании вы сможете немного расслабиться после реализации случайного леса и градиентного бустинга по мотивам материалов прошлой недели. Все, что нужно будет делать — запускать методы из sklearn. Вам предлагается выяснить, какое распределение лучше использовать в наивном байесовском классификаторе в зависимости от вида признаков.

In [1]:
# для записи ответов
def write_answer(answer, file_name):
    with open(file_name, 'w') as fout:
        fout.write(answer)

Загрузите датасеты digits и breast_cancer из sklearn.datasets.

Выведите несколько строчек из обучающих выборок и посмотрите на признаки.

In [15]:
from sklearn.datasets import load_digits, load_breast_cancer
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB

import numpy as np

In [10]:
digits = load_digits()
cancer = load_breast_cancer()

In [9]:
digits_X = digits.data
digits_y = digits.target
print(digits_X[:5])
print()
print(digits_y[:5])

[[ 0.  0.  5. 13.  9.  1.  0.  0.  0.  0. 13. 15. 10. 15.  5.  0.  0.  3.
  15.  2.  0. 11.  8.  0.  0.  4. 12.  0.  0.  8.  8.  0.  0.  5.  8.  0.
   0.  9.  8.  0.  0.  4. 11.  0.  1. 12.  7.  0.  0.  2. 14.  5. 10. 12.
   0.  0.  0.  0.  6. 13. 10.  0.  0.  0.]
 [ 0.  0.  0. 12. 13.  5.  0.  0.  0.  0.  0. 11. 16.  9.  0.  0.  0.  0.
   3. 15. 16.  6.  0.  0.  0.  7. 15. 16. 16.  2.  0.  0.  0.  0.  1. 16.
  16.  3.  0.  0.  0.  0.  1. 16. 16.  6.  0.  0.  0.  0.  1. 16. 16.  6.
   0.  0.  0.  0.  0. 11. 16. 10.  0.  0.]
 [ 0.  0.  0.  4. 15. 12.  0.  0.  0.  0.  3. 16. 15. 14.  0.  0.  0.  0.
   8. 13.  8. 16.  0.  0.  0.  0.  1.  6. 15. 11.  0.  0.  0.  1.  8. 13.
  15.  1.  0.  0.  0.  9. 16. 16.  5.  0.  0.  0.  0.  3. 13. 16. 16. 11.
   5.  0.  0.  0.  0.  3. 11. 16.  9.  0.]
 [ 0.  0.  7. 15. 13.  1.  0.  0.  0.  8. 13.  6. 15.  4.  0.  0.  0.  2.
   1. 13. 13.  0.  0.  0.  0.  0.  2. 15. 11.  1.  0.  0.  0.  0.  0.  1.
  12. 12.  1.  0.  0.  0.  0.  0.  1. 10.  8.  0.  0.  0.

In [14]:
cancer_X = cancer.data
cancer_y = cancer.target
print(cancer_X[:5])
print()
print(cancer_y[:5])

[[1.799e+01 1.038e+01 1.228e+02 1.001e+03 1.184e-01 2.776e-01 3.001e-01
  1.471e-01 2.419e-01 7.871e-02 1.095e+00 9.053e-01 8.589e+00 1.534e+02
  6.399e-03 4.904e-02 5.373e-02 1.587e-02 3.003e-02 6.193e-03 2.538e+01
  1.733e+01 1.846e+02 2.019e+03 1.622e-01 6.656e-01 7.119e-01 2.654e-01
  4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 1.326e+03 8.474e-02 7.864e-02 8.690e-02
  7.017e-02 1.812e-01 5.667e-02 5.435e-01 7.339e-01 3.398e+00 7.408e+01
  5.225e-03 1.308e-02 1.860e-02 1.340e-02 1.389e-02 3.532e-03 2.499e+01
  2.341e+01 1.588e+02 1.956e+03 1.238e-01 1.866e-01 2.416e-01 1.860e-01
  2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 1.203e+03 1.096e-01 1.599e-01 1.974e-01
  1.279e-01 2.069e-01 5.999e-02 7.456e-01 7.869e-01 4.585e+00 9.403e+01
  6.150e-03 4.006e-02 3.832e-02 2.058e-02 2.250e-02 4.571e-03 2.357e+01
  2.553e+01 1.525e+02 1.709e+03 1.444e-01 4.245e-01 4.504e-01 2.430e-01
  3.613e-01 8.758e-02]
 [1.142e+01 2.038e+01 7.758e+01 3.861e+02 1.425e-01 2.839e-01 2.414

С помощью sklearn.cross_validation.cross_val_score c настройками по умолчанию и вызова метода mean() у возвращаемого этой функцией numpy.ndarray, сравните качество работы наивных байесовских классификаторов на этих двух датасетах. Для сравнения предлагается использовать BernoulliNB, MultinomialNB и GaussianNB. Насколько полученные результаты согласуются с рекомендациями из лекций?

Два датасета, конечно, еще не повод делать далеко идущие выводы, но при желании вы можете продолжить исследование на других выборках (например, из UCI репозитория).

In [16]:
Bern = BernoulliNB()
Mult = MultinomialNB()
Gaus = GaussianNB()

In [30]:
print('           ', 'Digits            ', 'Breast Cancer')
print('Bernulli   ', cross_val_score(Bern, digits_X, digits_y).mean(), cross_val_score(Bern, cancer_X, cancer_y).mean())
print('Multinomial', cross_val_score(Mult, digits_X, digits_y).mean(), cross_val_score(Mult, cancer_X, cancer_y).mean())
print('Gaussian   ', cross_val_score(Gaus, digits_X, digits_y).mean(), cross_val_score(Gaus, cancer_X, cancer_y).mean())

            Digits             Breast Cancer
Bernulli    0.8241736304549674 0.6274181027790716
Multinomial 0.8703497369235531 0.8963204471355379
Gaussian    0.8069281956050759 0.9385188635305075


Для сдачи задания, ответьте на приведенные ниже вопросы.

Вопрос 1

Каким получилось максимальное качество классификации на датасете breast_cancer?

Вопрос 2

Каким получилось максимальное качество классификации на датасете digits?

Вопрос 3

Выберите верные утверждения и запишите их номера через пробел (в порядке возрастания номера):

1) На вещественных признаках лучше всего сработал наивный байесовский классификатор с распределением Бернулли

2) На вещественных признаках лучше всего сработал наивный байесовский классификатор с мультиномиальным распределением

3) Мультиномиальное распределение лучше показало себя на выборке с целыми неотрицательными значениями признаков

4) На вещественных признаках лучше всего сработало нормальное распределение

In [33]:
ans1 = str(cross_val_score(Gaus, cancer_X, cancer_y).mean())
ans2 = str(cross_val_score(Mult, digits_X, digits_y).mean())
ans3 = '3 4'
print(ans1, ans2, ans3, sep='\n')

write_answer(ans1, 'NB_families_ans1.txt')
write_answer(ans2, 'NB_families_ans2.txt')
write_answer(ans3, 'NB_families_ans3.txt')

0.9385188635305075
0.8703497369235531
3 4
